In [1]:
import argparse
import json
import tensorflow as tf
import numpy as np
import h5py
import copy
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
from evtk import hl

from fr3D.train.utils import setup_datasets
from fr3D.data.utils import get_normalization
from fr3D.models import ConvAutoencoder, ConvAutoencoderCGAN, ConvAutoencoderC

tf.keras.backend.set_image_data_format('channels_last')

def check_in_polygon(pts, vertices):
    path = mplPath.Path(vertices)
    return path.contains_points(pts)

def mape_with_threshold(yp, yt, pcterror_threshold=np.inf, max_magnitude_threshold=0.0, eps=1e-7):
    pct_errors = 100*tf.abs((yp-yt)/(eps + yt))
    pcterror_mask = pct_errors < pcterror_threshold
    max_magnitude_mask = tf.logical_not(tf.abs(yt) < (max_magnitude_threshold*tf.reduce_max(tf.abs(yt))))
    filtering_indices = tf.where(tf.logical_and(pcterror_mask, max_magnitude_mask))
    filtered_pcterrors = tf.gather_nd(pct_errors, filtering_indices)
    return float(tf.reduce_mean(filtered_pcterrors))

def get_normalization_type(node_configs):
    normalization_spec = {'method': None}
    for node in node_configs:
        if node['nodetype'] == 'normalize':
            normalization_spec = node['normalization_spec']
            break
    normalizer = get_normalization(**normalization_spec, batch_mode=True)
    return normalizer

def export_vtk(path, c, **fields):
    x = c[...,0].reshape(-1)
    y = c[...,1].reshape(-1)
    z = c[...,2].reshape(-1)
    data = {}
    for f in fields:
        data[f] = fields[f].reshape(-1).copy()
    hl.pointsToVTK(path, x, y, z, data = data)

In [2]:
Re = 500
expt_variables = ['U', 'V', 'W']

dataset_path = f'/fr3D/postprocessed/annulus_64_{Re}.h5'

#ConvAutoencoderC
experiment_configs = {expt_variable:f'/fr3D/configs/training/ConvAutoencoderC_{expt_variable}.json' for expt_variable in expt_variables}
weights_paths = {expt_variable:f'/storage/weights{Re}/ConvAutoencoderC_{expt_variable}_Annulus64/ConvAutoencoderC_{expt_variable}_Annulus64.h5' for expt_variable in expt_variables}



datasetf = h5py.File(dataset_path,'r')

shuf_buf = 1

train_datasets = {}
test_datasets = {}
sensor_shapes = {}
full_field_shapes = {}
normalizers = {}

for expt_variable in experiment_configs:
    config = json.load(open(experiment_configs[expt_variable],'r'))
    train_datasets[expt_variable], test_datasets[expt_variable] = setup_datasets(config, dataset_path, shuf_buf, case_names=True, evaluation=True)
    sensor_shapes[expt_variable] = train_datasets[expt_variable].element_spec[0][0].shape
    full_field_shapes[expt_variable] = train_datasets[expt_variable].element_spec[0][1].shape
    normalizers[expt_variable] = get_normalization_type(config['dataset']['node_configurations'])
    
dataset_iterators = {v: iter(test_datasets[v]) for v in test_datasets}

Training geometries: ('shape_69', 'shape_329', 'shape_46', 'shape_241', 'shape_239', 'shape_228', 'shape_163', 'shape_63', 'shape_137', 'shape_102', 'shape_157', 'shape_197', 'shape_52', 'shape_213', 'shape_84', 'shape_113', 'shape_58', 'shape_59', 'shape_146', 'shape_206', 'shape_331', 'shape_8', 'shape_18', 'shape_236', 'shape_43', 'shape_15', 'shape_23', 'shape_92', 'shape_135', 'shape_50', 'shape_14', 'shape_255', 'shape_75', 'shape_183', 'shape_89', 'shape_12', 'shape_374', 'shape_37', 'shape_218', 'shape_150', 'shape_31', 'shape_216', 'shape_13', 'shape_90', 'shape_182', 'shape_159', 'shape_67', 'shape_170', 'shape_358', 'shape_147', 'shape_17', 'shape_110', 'shape_152', 'shape_224', 'shape_130', 'shape_60', 'shape_126', 'shape_369', 'shape_22', 'shape_83', 'shape_338', 'shape_97', 'shape_33', 'shape_237', 'shape_325', 'shape_39', 'shape_244', 'shape_41', 'shape_355', 'shape_94', 'shape_61', 'shape_234', 'shape_34', 'shape_29', 'shape_95', 'shape_339', 'shape_55', 'shape_21', 'sh

2022-12-01 22:07:57.140194: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-12-01 22:07:57.140430: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2022-12-01 22:07:57.317518: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 22:07:58.190030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38258 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0
2022-12-01 22:07:58.191543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] 

Training geometries: ('shape_69', 'shape_329', 'shape_46', 'shape_241', 'shape_239', 'shape_228', 'shape_163', 'shape_63', 'shape_137', 'shape_102', 'shape_157', 'shape_197', 'shape_52', 'shape_213', 'shape_84', 'shape_113', 'shape_58', 'shape_59', 'shape_146', 'shape_206', 'shape_331', 'shape_8', 'shape_18', 'shape_236', 'shape_43', 'shape_15', 'shape_23', 'shape_92', 'shape_135', 'shape_50', 'shape_14', 'shape_255', 'shape_75', 'shape_183', 'shape_89', 'shape_12', 'shape_374', 'shape_37', 'shape_218', 'shape_150', 'shape_31', 'shape_216', 'shape_13', 'shape_90', 'shape_182', 'shape_159', 'shape_67', 'shape_170', 'shape_358', 'shape_147', 'shape_17', 'shape_110', 'shape_152', 'shape_224', 'shape_130', 'shape_60', 'shape_126', 'shape_369', 'shape_22', 'shape_83', 'shape_338', 'shape_97', 'shape_33', 'shape_237', 'shape_325', 'shape_39', 'shape_244', 'shape_41', 'shape_355', 'shape_94', 'shape_61', 'shape_234', 'shape_34', 'shape_29', 'shape_95', 'shape_339', 'shape_55', 'shape_21', 'sh

In [3]:
models = {}

for v in expt_variables:
    config = json.load(open(experiment_configs[v],'r'))
    model = ConvAutoencoderC(dense_input_units=sensor_shapes[v][1],
                             autoencoder_input_shape=full_field_shapes[v][1:],
                             **config['model'])
    loss_fn = "mse"#tf.keras.losses.get(config['training']['loss'])
    model.compile(l_optimizer= tf.keras.optimizers.get(config['training']['l_optimizer']),
                  loss=loss_fn,
                  optimizer = tf.keras.optimizers.get(config['training']['ae_optimizer']),
                  metrics = config['training'].get('metrics', None))
    model.load_weights(weights_paths[v])

In [4]:
idx = 0

inps = {}
targets = {}
norm_params = {}
case_names = {}
preds = {}
ae_preds = {}

for v in expt_variables:    
    (inp, target, norm_param), case_name = next(dataset_iterators[v])
    pred = model(inp,autoencode=False)
    ae_pred = model(target,autoencode=True)
    
    ntarget = normalizers[v].undo(target, norm_param[:,0,:])
    npred = normalizers[v].undo(pred, norm_param[:,0,:])
    nae_pred = normalizers[v].undo(ae_pred, norm_param[:,0,:])
    
    import pdb; pdb.set_trace()

    inps[v] = inp[idx]
    targets['gt'+v] = ntarget[idx,...,0].numpy()
    norm_params[v] = norm_param[idx]
    case_names[v] = case_name[idx].numpy()
    preds['pred'+v] = npred[idx,...,0].numpy()
    ae_preds['aepred'+v] = nae_pred[idx,...,0].numpy()

assert len(set(case_names.values()))==1
case_name = list(case_names.values())[0]
coords = datasetf[case_name]['full_field_coords']

export_vtk(f"/storage/qcrit_{case_name.decode()}", coords, **targets, **ae_preds, **preds)

2022-12-01 22:08:30.922668: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-12-01 22:08:31.333129: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-12-01 22:08:31.807134: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


> /tmp/ipykernel_383462/2599994667.py(21)<cell line: 10>()
     19     import pdb; pdb.set_trace()
     20 
---> 21     inps[v] = inp[idx]
     22     targets['gt'+v] = target[idx,...,0].numpy()
     23     norm_params[v] = norm_param[idx]

ipdb> print(ntarget.shape)
(24, 64, 64, 64, 1)
ipdb> q


In [4]:
normalizers[0]

{'U': <fr3D.data.utils.normalization.MinMaxNormalization at 0x7fb70c1840d0>,
 'V': <fr3D.data.utils.normalization.MinMaxNormalization at 0x7fb680133fd0>,
 'W': <fr3D.data.utils.normalization.MinMaxNormalization at 0x7fb62823b100>}